# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [1]:
# Atualmente, este é meu bloco fixo de importações
import pandas as pd
import seaborn as sns
from seaborn import load_dataset
import numpy as np
import matplotlib.pyplot as plt
import requests as rq
# Bloco de estudo estatístico
import statsmodels.api as sm
import statsmodels.formula.api as snf
from sklearn.metrics import r2_score
# Modelando com o Patsy
import patsy

In [2]:
df = pd.read_csv("previsao_de_renda.csv")

In [3]:
print(df.columns)
df.info()

Index(['Unnamed: 0', 'data_ref', 'id_cliente', 'sexo', 'posse_de_veiculo',
       'posse_de_imovel', 'qtd_filhos', 'tipo_renda', 'educacao',
       'estado_civil', 'tipo_residencia', 'idade', 'tempo_emprego',
       'qt_pessoas_residencia', 'renda'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  ob

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.  


2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.  


3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

In [4]:
# Modelando com os recursos do Patsy
construc_mod = "np.log(renda) ~ C(sexo) + posse_de_veiculo + posse_de_imovel + qtd_filhos + C(tipo_renda) + C(educacao) + C(estado_civil) + C(tipo_residencia) + idade + tempo_emprego + qt_pessoas_residencia"

y, X = patsy.dmatrices(construc_mod, data = df, return_type = "dataframe")
mod_lnrenda = sm.OLS(y, X).fit()
print(mod_lnrenda.summary())

# Obtendo os p-valores das variaveis independentes, garantindo o formato da entrada para comparacao de significancia
p_values = mod_lnrenda.summary().tables[1].data[1:]
p_values = [float(row[4]) for row in p_values]

# Extraindo a tabela sem contabilizar as informacoes do cabeçalho
significantes = mod_lnrenda.summary().tables[1].data[1:]

# Filtrando para uma lista através pelas linhas de acordo com sua significancia de 5%
significantes = [row[0] for row in significantes if float(row[4]) < 0.05]

significantes

                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     287.5
Date:                Tue, 15 Aug 2023   Prob (F-statistic):               0.00
Time:                        18:29:22   Log-Likelihood:                -13568.
No. Observations:               12427   AIC:                         2.719e+04
Df Residuals:                   12402   BIC:                         2.737e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
In

['Intercept',
 'C(sexo)[T.M]',
 'posse_de_veiculo[T.True]',
 'posse_de_imovel[T.True]',
 'C(tipo_renda)[T.Empresário]',
 'C(tipo_renda)[T.Servidor público]',
 'C(estado_civil)[T.Separado]',
 'C(estado_civil)[T.Solteiro]',
 'C(estado_civil)[T.Viúvo]',
 'qtd_filhos',
 'idade',
 'tempo_emprego',
 'qt_pessoas_residencia']

In [7]:
# Remodelando com os recursos do Patsy
teste1_mod = "np.log(renda) ~ C(sexo) + posse_de_veiculo + posse_de_imovel + qtd_filhos + C(tipo_renda) + C(estado_civil) + idade + tempo_emprego + qt_pessoas_residencia"

y, X = patsy.dmatrices(teste1_mod, data = df, return_type = "dataframe")
mod2_lnrenda = sm.OLS(y, X).fit()
print(mod2_lnrenda.summary())

# Obtendo os p-valores das variaveis independentes, garantindo o formato da entrada para comparacao de significancia
p_values = mod2_lnrenda.summary().tables[1].data[1:]
p_values = [float(row[4]) for row in p_values]

significantes2 = mod2_lnrenda.summary().tables[1].data[1:] # Extraindo a tabela sem contabilizar as informacoes do cabeçalho

# Filtrando para uma lista através pelas linhas de acordo com sua significancia de 5%
significantes2 = [row[0] for row in significantes2 if float(row[4]) < 0.05]

significantes2

                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.354
Model:                            OLS   Adj. R-squared:                  0.353
Method:                 Least Squares   F-statistic:                     453.1
Date:                Tue, 15 Aug 2023   Prob (F-statistic):               0.00
Time:                        18:33:18   Log-Likelihood:                -13603.
No. Observations:               12427   AIC:                         2.724e+04
Df Residuals:                   12411   BIC:                         2.736e+04
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Interc

['Intercept',
 'C(sexo)[T.M]',
 'posse_de_veiculo[T.True]',
 'posse_de_imovel[T.True]',
 'C(tipo_renda)[T.Empresário]',
 'C(tipo_renda)[T.Servidor público]',
 'C(estado_civil)[T.Separado]',
 'C(estado_civil)[T.Solteiro]',
 'C(estado_civil)[T.Viúvo]',
 'qtd_filhos',
 'idade',
 'tempo_emprego',
 'qt_pessoas_residencia']

In [8]:
# Remodelando com os recursos do Patsy
teste2_mod = "np.log(renda) ~ C(sexo) + posse_de_veiculo + posse_de_imovel + qtd_filhos + idade + tempo_emprego + qt_pessoas_residencia"

y, X = patsy.dmatrices(teste2_mod, data = df, return_type = "dataframe")
mod3_lnrenda = sm.OLS(y, X).fit()
print(mod3_lnrenda.summary())

# Obtendo os p-valores das variaveis independentes, garantindo o formato da entrada para comparacao de significancia
p_values = mod3_lnrenda.summary().tables[1].data[1:]
p_values = [float(row[4]) for row in p_values]

significantes3 = mod3_lnrenda.summary().tables[1].data[1:] # Extraindo a tabela sem contabilizar as informacoes do cabeçalho

# Filtrando para uma lista através pelas linhas de acordo com sua significancia de 5%
significantes3 = [row[0] for row in significantes3 if float(row[4]) < 0.05]

significantes3

                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.347
Model:                            OLS   Adj. R-squared:                  0.346
Method:                 Least Squares   F-statistic:                     940.8
Date:                Tue, 15 Aug 2023   Prob (F-statistic):               0.00
Time:                        18:34:18   Log-Likelihood:                -13672.
No. Observations:               12427   AIC:                         2.736e+04
Df Residuals:                   12419   BIC:                         2.742e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept               

['Intercept',
 'C(sexo)[T.M]',
 'posse_de_veiculo[T.True]',
 'posse_de_imovel[T.True]',
 'idade',
 'tempo_emprego']

In [11]:
# Remodelando com os recursos do Patsy
teste3_mod = "np.log(renda) ~ C(sexo) + posse_de_veiculo + posse_de_imovel + idade + tempo_emprego"

y, X = patsy.dmatrices(teste3_mod, data = df, return_type = "dataframe")
mod3_lnrenda = sm.OLS(y, X).fit()
print(mod3_lnrenda.summary())

# Obtendo os p-valores das variaveis independentes, garantindo o formato da entrada para comparacao de significancia
p_values = mod3_lnrenda.summary().tables[1].data[1:]
p_values = [float(row[4]) for row in p_values]

significantes4 = mod3_lnrenda.summary().tables[1].data[1:] # Extraindo a tabela sem contabilizar as informacoes do cabeçalho

# Filtrando para uma lista através pelas linhas de acordo com sua significancia de 5%
significantes4 = [row[0] for row in significantes4 if float(row[4]) < 0.05]

significantes4

                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.346
Model:                            OLS   Adj. R-squared:                  0.346
Method:                 Least Squares   F-statistic:                     1315.
Date:                Tue, 15 Aug 2023   Prob (F-statistic):               0.00
Time:                        18:37:20   Log-Likelihood:                -13676.
No. Observations:               12427   AIC:                         2.736e+04
Df Residuals:                   12421   BIC:                         2.741e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept               

['Intercept',
 'C(sexo)[T.M]',
 'posse_de_veiculo[T.True]',
 'posse_de_imovel[T.True]',
 'idade',
 'tempo_emprego']

Apesar do R² reduzir um pouco [de 0,357 para 0,346], aparentemente os outros parâmetros ficaram melhores relativamente ao inicio.